In [3]:
# ─────────────────────────────────────────────────────────────
# Imports
# ─────────────────────────────────────────────────────────────
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import numpy as np
import os

# ─────────────────────────────────────────────────────────────
# Load Exposure Points
# ─────────────────────────────────────────────────────────────
exposure_path = r"C:\CMIP data\GitHub Clean\ccart-cyclone-impact-modeling\outputs\exposure_points.geojson"
gdf = gpd.read_file(exposure_path)

# ─────────────────────────────────────────────────────────────
# Define Raster Parameters
# ─────────────────────────────────────────────────────────────
res = 0.01  # degrees per pixel (adjust for resolution)
xmin, ymin, xmax, ymax = gdf.total_bounds
width = int((xmax - xmin) / res)
height = int((ymax - ymin) / res)
transform = from_origin(xmin, ymax, res, res)

# ─────────────────────────────────────────────────────────────
# Prepare Shapes for Rasterization
# ─────────────────────────────────────────────────────────────
shapes = [
    (geom, np.log10(max(val, 1)))  # log₁₀(asset value)
    for geom, val in zip(gdf.geometry, gdf["value"])
]

# ─────────────────────────────────────────────────────────────
# Rasterize
# ─────────────────────────────────────────────────────────────
raster = rasterize(
    shapes,
    out_shape=(height, width),
    transform=transform,
    fill=np.nan,
    dtype="float32"
)

raster = np.log10(np.maximum(raster, 1))

# ─────────────────────────────────────────────────────────────
# Save as GeoTIFF
# ─────────────────────────────────────────────────────────────
output_path = r"C:\CMIP data\GitHub Clean\ccart-cyclone-impact-modeling\outputs\exposure_raster.tif"
with rasterio.open(
    output_path, "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype=raster.dtype,
    crs="EPSG:4326",
    transform=transform,
    nodata=np.nan
) as dst:
    dst.write(raster, 1)

print("✅ Rasterization complete. GeoTIFF saved to:")
print(output_path)


✅ Rasterization complete. GeoTIFF saved to:
C:\CMIP data\GitHub Clean\ccart-cyclone-impact-modeling\outputs\exposure_raster.tif
